In [120]:
from WindPy import w
from datetime import datetime, timedelta
w.start()
date = '2022-11-18'
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
w.wsd(code[0], "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y", '2022-11-5', '2022-11-18', "annualized=1", usedf=True)[1]

,SEC_NAME,NAV,NAV_ADJ_RETURN1,RETURN_1M,RETURN_3M,RETURN_1Y
2022-11-07,平安短债A,1.1488,0.008705,2.492929,1.922233,3.170184
2022-11-08,平安短债A,1.1486,-0.017409,2.283034,1.710601,3.124439
2022-11-09,平安短债A,1.1485,-0.008706,2.178233,1.710751,3.124719
2022-11-10,平安短债A,1.1482,-0.026121,1.031175,1.570145,3.070018
2022-11-11,平安短债A,1.1478,-0.034837,0.514333,1.500236,3.034111
2022-11-14,平安短债A,1.1463,-0.130685,-1.325670,0.974999,2.890225
2022-11-15,平安短债A,1.1453,-0.087237,-2.334451,0.416772,2.791240
2022-11-16,平安短债A,1.1443,-0.087313,-3.333787,-0.034663,2.683058
2022-11-17,平安短债A,1.1436,-0.061173,-4.420734,-0.277055,2.601830
2022-11-18,平安短债A,1.1438,0.017489,-4.321913,-0.242434,2.601363


In [110]:
from typing import Literal
from WindPy import w
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class Wind_Exporter:
    def __init__(self, code:str|list, indicator:str, method:Literal['wsd','wss']='wsd', Date_List:list=None, StartDate=None, EndDate=None, options=None):
        self.method = method
        self.code = code
        self.EndDate = EndDate if EndDate else datetime.today().strftime('%Y-%m-%d')
        self.Date_List = Date_List
        self.StartDate = EndDate if StartDate is None else StartDate
        self.indicator = indicator
        self.options = options
        self.data = []

    def check_connection(func):
        if not w.isconnected():
            w.start()
        return func

    @staticmethod
    def create_date_col(df, date):
        df['date'] = date
        column = np.roll(np.arange(len(df.columns)), 1)
        column[0], column[1] =column[1], column[0]
        return df.iloc[:, column]

    @check_connection
    def get_data(self, output:Literal[None,'df','excel']=None, round=None):
        if self.method == 'wsd':
            self.get_data_wsd()
        elif self.method == 'wss':
            self.get_data_wss()
        else:
            raise ValueError('method must be wsd or wss')


        if round is not None:
            for i in range(len(self.data)):
                self.data[i] = self.data[i].round(round)

        
        if output == 'df':
            return self.data
        elif output == 'excel':
            pass
        else:
            return self

    def get_data_wsd(self):
        if self.Date_List is None:
            if self.StartDate == self.EndDate:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, self.StartDate, self.EndDate, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                df = self.create_date_col(pd.concat(multi_data, axis=0), self.EndDate) 
                self.data.append(df)
            else:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, self.StartDate, self.EndDate, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                self.data.append(multi_data)
        else:
            for date in self.Date_List:
                multi_data = []
                for i in self.code:
                    data_ = w.wsd(i, self.indicator, date, date, options=self.options, usedf=True)
                    multi_data.append(data_[1])
                df = self.create_date_col(pd.concat(multi_data, axis=0), date)
                self.data.append(df)
        return self
    def get_data_wss(self):
        pass


    def excel_export(self, path=None, sheet_name:list=None, column_name:list=None):
        if path is None:
            if self.Date_List is None:
                if self.StartDate == self.EndDate:
                    path = f'./output/{self.EndDate}.xlsx'
                else:
                    path = f'./output/{self.StartDate}_{self.EndDate}.xlsx'
            else:
                path = f'./output/{self.Date_List[0]}.xlsx'
        if sheet_name is None:
            sheet_name = np.arange(len(self.data)) + 1
        if column_name is not None:
            for i in range(len(self.data)):
                self.data[i].columns = column_name
        with pd.ExcelWriter(path) as writer:
            for i in range(len(self.data)):
                self.data[i].to_excel(writer, sheet_name=f'{sheet_name[i]}')
        return self
    def df_export(self):
        return self.data
        


indicator = "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y"
date = ['2022-11-18', '2022-11-17']
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
Wind_Exporter(code, method='wsd', indicator=indicator, EndDate='2022-11-18', options="annualized=1").get_data(round=2).excel_export()
# wsd_parser(code, indicator=indicator, Date_List=date, options="annualized=1").get_data()

code_for_meiri_str = """002450.OF,004827.OF,015645.OF,008694.OF,005754.OF,700003.OF,000739.OF,007935.OF,009661.OF,009878.OF,010126.OF,014460.OF,013767.OF,013687.OF,004390.OF,012475.OF,007893.OF,885001.WI"""
data_meiri = w.wss(code_for_meiri_str, "sec_name,nav,NAV_adj_return1,NAV_adj_return,return_ytd",f"tradeDate={end_date_};startDate={date_before_1m};endDate={end_date_};annualized=0")



In [132]:
from WdpCore import Wind_Exporter
indicator = "sec_name,nav,NAV_adj_return1,return_1m,return_3m,return_1y"
date = ['2022-11-18', '2022-11-17']
code = ['005754.OF', '005756.OF', '008911.OF', '008913.OF', '007935.OF', '007936.OF', '008696.OF', '004827.OF', '006851.OF']
Wind_Exporter(code, method='wsd', indicator=indicator, Date_List=date, options="annualized=1").get_data(round=2).excel_export(sheet_name=['666', '777'])

In [99]:
for i in range(2):
    print(i)

0
1


In [126]:
code_for_meiri_str = """002450.OF,004827.OF,015645.OF,008694.OF,005754.OF,700003.OF,000739.OF,007935.OF,009661.OF,009878.OF,010126.OF,014460.OF,013767.OF,013687.OF,004390.OF,012475.OF,007893.OF,885001.WI"""
# w.wss(code_for_meiri_str, "sec_name,nav,NAV_adj_return1,NAV_adj_return,return_ytd","tradeDate='2022-11-18';startDate='2022-10-19';endDate='2022-11-18';annualized=0")
w.wss("002450.OF","NAV_adj_return","startDate=20221117;endDate=20221118")
# w.wss("009878.OF", "sec_name")

.ErrorCode=0
.Codes=[002450.OF]
.Fields=[NAV_ADJ_RETURN]
.Times=[20221119 21:30:14]
.Data=[[-1.1076650420912688]]

In [103]:
output = Wind_Exporter(code, method='wsd', indicator=indicator, EndDate='2022-11-18', options="annualized=1").get_data().data


In [106]:
with pd.ExcelWriter('./output/2022-11-18.xlsx') as writer:
    for i in range(len(output)):
        output[i].to_excel(writer,sheet_name=i+1)

TypeError: expected string or bytes-like object

In [ ]:
pd.DateOffset